In this lecture we're going to review some of the basics of statistical testing in python. We're going to
talk about hypothesis testing, statistical significance, and using scipy to run student's t-tests.

In [1]:
# We use statistics in a lot of different ways in data science, 
# and on this lecture, I want to refresh your
# knowledge of hypothesis testing, which is a core data analysis 
# activity behind experimentation. The goal of
# hypothesis testing is to determine if, for instance, 
# the two different conditions we have in an experiment 
# have resulted in different impacts

# Let's import our usual numpy and pandas libraries
import numpy as np
import pandas as pd

# Now let's bring in some new libraries from scipy
from scipy import stats

In [2]:
# Now, scipy is an interesting collection of libraries for data 
# science and you'll use most or perpahs all of
# these libraries. It includes numpy and pandas,
# but also plotting libraries such as matplotlib, and a
# number of scientific library functions as well

In [2]:
# When we do hypothesis testing, 
# we actually have two statements of interest: the first is our actual
# explanation, which we call the alternative hypothesis,
# and the second is that the explanation we have is not
# sufficient, and we call this the null hypothesis. 
# Our actual testing method is to determine whether the null
# hypothesis is true or not.
# If we find that there is a difference between groups,
# then we can reject the null
# hypothesis and we accept our alternative.

# Let's see an example of this; we're going to use some grade data
df=pd.read_csv ('datasets/grades.csv')
df.head(5)

,student_id,assignment1_grade,assignment1_submission,assignment2_grade,assignment2_submission,assignment3_grade,assignment3_submission,assignment4_grade,assignment4_submission,assignment5_grade,assignment5_submission,assignment6_grade,assignment6_submission
0,B73F2C11-70F0-E37D-8B10-1D20AFED50B1,92.733946,2015-11-02 06:55:34.282000000,83.030552,2015-11-09 02:22:58.938000000,67.164441,2015-11-12 08:58:33.998000000,53.011553,2015-11-16 01:21:24.663000000,47.710398,2015-11-20 13:24:59.692000000,38.168318,2015-11-22 18:31:15.934000000
1,98A0FAE0-A19A-13D2-4BB5-CFBFD94031D1,86.790821,2015-11-29 14:57:44.429000000,86.290821,2015-12-06 17:41:18.449000000,69.772657,2015-12-10 08:54:55.904000000,55.098125,2015-12-13 17:32:30.941000000,49.588313,2015-12-19 23:26:39.285000000,44.629482,2015-12-21 17:07:24.275000000
2,D0F62040-CEB0-904C-F563-2F8620916C4E,85.512541,2016-01-09 05:36:02.389000000,85.512541,2016-01-09 06:39:44.416000000,68.410033,2016-01-15 20:22:45.882000000,54.728026,2016-01-11 12:41:50.749000000,49.255224,2016-01-11 17:31:12.489000000,44.329701,2016-01-17 16:24:42.765000000
3,FFDF2B2C-F514-EF7F-6538-A6A53518E9DC,86.030665,2016-04-30 06:50:39.801000000,68.824532,2016-04-30 17:20:38.727000000,61.942079,2016-05-12 07:47:16.326000000,49.553663,2016-05-07 16:09:20.485000000,49.553663,2016-05-24 12:51:18.016000000,44.598297,2016-05-26 08:09:12.058000000
4,5ECBEEB6-F1CE-80AE-3164-E45E99473FB4,64.813800,2015-12-13 17:06:10.750000000,51.491040,2015-12-14 12:25:12.056000000,41.932832,2015-12-29 14:25:22.594000000,36.929549,2015-12-28 01:29:55.901000000,33.236594,2015-12-29 14:46:06.628000000,33.236594,2016-01-05 01:06:59.546000000


In [3]:
# If we take a look at the data frame inside,
# we see we have six different assignments. Lets look at some
# summary statistics for this DataFrame

print("There are {} rows and {} columns".format(df.shape[0], df.shape[1]))

There are 2315 rows and 13 columns


In [4]:
# For the purpose of this lecture, 
# let's segment this population into two pieces. Let's say those who finish
# the first assignment by the end of December 2015,
# we'll call them early finishers, and those who finish it 
# sometime after that, we'll call them late finishers.

early_finishers=df[pd.to_datetime(df['assignment1_submission']) < '2016']
early_finishers.head(50)

,student_id,assignment1_grade,assignment1_submission,assignment2_grade,assignment2_submission,assignment3_grade,assignment3_submission,assignment4_grade,assignment4_submission,assignment5_grade,assignment5_submission,assignment6_grade,assignment6_submission
0,B73F2C11-70F0-E37D-8B10-1D20AFED50B1,92.733946,2015-11-02 06:55:34.282000000,83.030552,2015-11-09 02:22:58.938000000,67.164441,2015-11-12 08:58:33.998000000,53.011553,2015-11-16 01:21:24.663000000,47.710398,2015-11-20 13:24:59.692000000,38.168318,2015-11-22 18:31:15.934000000
1,98A0FAE0-A19A-13D2-4BB5-CFBFD94031D1,86.790821,2015-11-29 14:57:44.429000000,86.290821,2015-12-06 17:41:18.449000000,69.772657,2015-12-10 08:54:55.904000000,55.098125,2015-12-13 17:32:30.941000000,49.588313,2015-12-19 23:26:39.285000000,44.629482,2015-12-21 17:07:24.275000000
4,5ECBEEB6-F1CE-80AE-3164-E45E99473FB4,64.813800,2015-12-13 17:06:10.750000000,51.491040,2015-12-14 12:25:12.056000000,41.932832,2015-12-29 14:25:22.594000000,36.929549,2015-12-28 01:29:55.901000000,33.236594,2015-12-29 14:46:06.628000000,33.236594,2016-01-05 01:06:59.546000000
5,D09000A0-827B-C0FF-3433-BF8FF286E15B,71.647278,2015-12-28 04:35:32.836000000,64.052550,2016-01-03 21:05:38.392000000,64.752550,2016-01-07 08:55:43.692000000,57.467295,2016-01-11 00:45:28.706000000,57.467295,2016-01-11 00:54:13.579000000,57.467295,2016-01-20 19:54:46.166000000
8,C9D51293-BD58-F113-4167-A7C0BAFCB6E5,66.595568,2015-12-25 02:29:28.415000000,52.916454,2015-12-31 01:42:30.046000000,48.344809,2016-01-05 23:34:02.180000000,47.444809,2016-01-02 07:48:42.517000000,37.955847,2016-01-03 21:27:04.266000000,37.955847,2016-01-19 15:24:31.060000000
11,B017B4A6-1F41-9D46-A040-21B0D270D4BE,76.670469,2015-10-05 02:11:05.947000000,68.573422,2015-10-11 23:08:01.288000000,69.273422,2015-10-14 06:03:04.482000000,54.698738,2015-10-19 00:34:45.875000000,49.228864,2015-10-24 11:25:20.846000000,49.228864,2015-10-26 15:56:55.374000000
15,24827B50-2FC2-E4B0-E2A0-1989E17749B3,14.959603,2015-11-17 01:16:07.800000000,14.459603,2015-11-22 15:00:14.342000000,12.307682,2015-11-28 13:12:23.939000000,9.126146,2015-11-29 17:06:51.153000000,8.213531,2015-12-06 21:21:09.208000000,7.392178,2015-12-10 01:16:09.387000000
17,D6EFCCCA-82CA-C4BE-1C3E-D0477DE28B24,53.555227,2015-09-17 13:06:39.955000000,53.055227,2015-09-20 09:57:00.973000000,48.469704,2015-09-21 13:17:53.140000000,47.569704,2015-09-22 10:12:42.156000000,42.812734,2015-09-27 10:58:56.037000000,42.812734,2015-09-30 18:24:47.681000000
19,C040D322-E970-4E8F-22A6-2FFAF2D74061,36.383575,2015-11-12 14:05:47.682000000,32.315217,2015-11-29 12:17:05.380000000,33.015217,2015-12-24 08:32:51.206000000,25.692174,2015-12-15 12:46:09.484000000,20.553739,2015-12-27 06:11:42.755000000,16.442991,2016-01-04 07:07:44.849000000
22,A1331E62-50B5-94FC-E0CE-3092CB3D30D9,88.901114,2015-09-16 22:33:02.686000000,70.760891,2015-09-23 23:30:17.970000000,64.404802,2015-09-30 03:54:05.524000000,50.803842,2015-10-02 00:38:00.096000000,50.803842,2015-10-08 18:57:57.501000000,45.723457,2015-10-10 20:42:09.120000000


In [13]:
# So, you have lots of skills now with pandas, 
# how would you go about getting the late_finishers dataframe?
# Why don't you pause the video and give it a try.

In [6]:
# Here's my solution. First, the dataframe df and the early_finishers
# share index values, so I really just
# want everything in the df which is not in early_finishers
# pues ser asi: nii = df[pd.to_datetime(df['assignment1_submission']) >= '2016']

#o bien,
late_finishers=df[~df.index.isin(early_finishers.index)]
late_finishers.head(5)


,student_id,assignment1_grade,assignment1_submission,assignment2_grade,assignment2_submission,assignment3_grade,assignment3_submission,assignment4_grade,assignment4_submission,assignment5_grade,assignment5_submission,assignment6_grade,assignment6_submission
2,D0F62040-CEB0-904C-F563-2F8620916C4E,85.512541,2016-01-09 05:36:02.389000000,85.512541,2016-01-09 06:39:44.416000000,68.410033,2016-01-15 20:22:45.882000000,54.728026,2016-01-11 12:41:50.749000000,49.255224,2016-01-11 17:31:12.489000000,44.329701,2016-01-17 16:24:42.765000000
3,FFDF2B2C-F514-EF7F-6538-A6A53518E9DC,86.030665,2016-04-30 06:50:39.801000000,68.824532,2016-04-30 17:20:38.727000000,61.942079,2016-05-12 07:47:16.326000000,49.553663,2016-05-07 16:09:20.485000000,49.553663,2016-05-24 12:51:18.016000000,44.598297,2016-05-26 08:09:12.058000000
6,3217BE3F-E4B0-C3B6-9F64-462456819CE4,87.498744,2016-03-05 11:05:25.408000000,69.998995,2016-03-09 07:29:52.405000000,55.999196,2016-03-16 22:31:24.316000000,50.399276,2016-03-18 07:19:26.032000000,45.359349,2016-03-19 10:35:41.869000000,45.359349,2016-03-23 14:02:00.987000000
7,F1CB5AA1-B3DE-5460-FAFF-BE951FD38B5F,80.576090,2016-01-24 18:24:25.619000000,72.518481,2016-01-27 13:37:12.943000000,65.266633,2016-01-30 14:34:36.581000000,65.266633,2016-02-03 22:08:49.002000000,65.266633,2016-02-16 14:22:23.664000000,65.266633,2016-02-18 08:35:04.796000000
9,E2C617C2-4654-622C-AB50-1550C4BE42A0,59.270882,2016-03-06 12:06:26.185000000,59.270882,2016-03-13 02:07:25.289000000,53.343794,2016-03-17 07:30:09.241000000,53.343794,2016-03-20 21:45:56.229000000,42.675035,2016-03-27 15:55:04.414000000,38.407532,2016-03-30 20:33:13.554000000


In [22]:
# There are lots of other ways to do this.
# For instance, you could just copy and paste the first projection
# and change the sign from less than to greater than or equal to. 
# This is ok, but if you decide you want to
# change the date down the road you have to remember to change it 
# in two places. You could also do a join of
# the dataframe df with early_finishers -
# if you do a left join you only keep the items in the left dataframe,
# so this would have been a good answer. 
# You also could have written a function that determines if someone is
# early or late, and then called .apply() on the dataframe
# and added a new column to the dataframe. This is a
# pretty reasonable answer as well.

In [9]:
# As you've seen, the pandas data frame object has a variety of statistical
# functions associated with it. If
# we call the mean function directly on the data frame,
# we see that each of the means for the assignments are
# calculated. Let's compare the means for our two populations
print(early_finishers.mean())
print(late_finishers.mean())

print(early_finishers['assignment1_grade'].mean())
print(late_finishers['assignment1_grade'].mean())

assignment1_grade    74.947285
assignment2_grade    67.229129
assignment3_grade    61.098805
assignment4_grade    54.126001
assignment5_grade    48.604524
assignment6_grade    43.812144
dtype: float64
assignment1_grade    74.045065
assignment2_grade    66.395813
assignment3_grade    60.056162
assignment4_grade    54.095552
assignment5_grade    48.635211
assignment6_grade    43.876394
dtype: float64
74.94728457024304
74.0450648477065


In [11]:
# Ok, these look pretty similar. But, are they the same? 
# What do we mean by similar? This is where the
# students' t-test comes in. It allows us to form
# the alternative hypothesis ("These are different") as well
# as the null hypothesis ("These are the same") and then test 
# that null hypothesis.

# When doing hypothesis testing, we have to choose a significance
# level as a threshold for how much of a
# chance we're willing to accept. This significance level is 
# typically called alpha. For this example, let's
# use a threshold of 0.05 for our alpha or 5%. Now this is a
# commonly used number but it's really quite
# arbitrary.

# The SciPy library contains a number of different statistical 
# tests and forms a basis for hypothesis testing
# in Python and we're going to use the ttest_ind() 
# function which does an independent t-test (meaning the
# populations are not related to one another).

# The result of ttest_index() are the t-statistic and a p-value.
# It's this latter value, the probability, which is most important to us,
# as it indicates the chance (between
# 0 and 1) of our null hypothesis being True.

# Let's bring in our ttest_ind function
from scipy.stats import ttest_ind

# Let's run this function with our two populations,
# looking at the assignment 1 grades

ttest_ind(early_finishers['assignment1_grade'], 
          late_finishers['assignment1_grade'])

Ttest_indResult(statistic=1.3223540853721596, pvalue=0.18618101101713855)

In [12]:
# So here we see that the probability is 0.18,
# and this is above our alpha value of 0.05. This means that we
# cannot reject the null hypothesis. 
# The null hypothesis was that the two populations are the same, and we
# don't have enough certainty in our evidence
# (because it is greater than alpha) to come to a conclusion to
# the contrary. This doesn't mean that we have proven 
# the populations are the same.

In [13]:
# Why don't we check the other assignment grades?
print(ttest_ind(early_finishers['assignment2_grade'], late_finishers['assignment2_grade']))
print(ttest_ind(early_finishers['assignment3_grade'], late_finishers['assignment3_grade']))
print(ttest_ind(early_finishers['assignment4_grade'], late_finishers['assignment4_grade']))
print(ttest_ind(early_finishers['assignment5_grade'], late_finishers['assignment5_grade']))
print(ttest_ind(early_finishers['assignment6_grade'], late_finishers['assignment6_grade']))

Ttest_indResult(statistic=1.2514717608216366, pvalue=0.2108889627004424)
Ttest_indResult(statistic=1.6133726558705392, pvalue=0.10679998102227865)
Ttest_indResult(statistic=0.049671157386456125, pvalue=0.960388729789337)
Ttest_indResult(statistic=-0.05279315545404755, pvalue=0.9579012739746492)
Ttest_indResult(statistic=-0.11609743352612056, pvalue=0.9075854011989656)


In [14]:
# Ok, so it looks like in this data we do not have enough
# evidence to suggest the populations differ with
# respect to grade. Let's take a look at those p-values 
# for a moment though, because they are saying things
# that can inform experimental design down the road. 
# For instance, one of the assignments, assignment 3, has a
# p-value around 0.1. 
# This means that if we accepted a level of chance similarity of 11% 
# this would have been
# considered statistically significant.
# As a research, this would suggest to me that there is something here
# worth considering following up on.

# For instance, if we had a small number of participants (we don't) or if
# there was something unique about this assignment 
# as it relates to our experiment (whatever it was) then
# there may be followup experiments we could run.

In [15]:
# P-values have come under fire recently for being insuficient
# for telling us enough about the interactions
# which are happening, and two other techniques, 
# confidence intervalues and bayesian analyses, are being used
# more regularly.

# One issue with p-values is that as you run more tests you 
# are likely to get a value which
# is statistically significant just by chance.

# Lets see a simulation of this. 
# First, lets create a data frame of 100 columns, each with 100 numbers
df1=pd.DataFrame([np.random.random(100) for x in range(100)])
df1.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,0.771981,0.300807,0.859511,0.527129,0.298103,0.765953,0.625381,0.311769,0.307172,0.416089,...,0.991853,0.711573,0.297490,0.554781,0.193479,0.936550,0.817640,0.363476,0.150649,0.590393
1,0.201107,0.996606,0.560926,0.194175,0.904290,0.406860,0.204319,0.285607,0.280851,0.652569,...,0.015991,0.709135,0.418303,0.694884,0.686240,0.683541,0.917331,0.271377,0.890925,0.671587
2,0.450912,0.910108,0.398643,0.667051,0.081442,0.551772,0.986264,0.022460,0.517304,0.811148,...,0.058834,0.977276,0.471014,0.827639,0.310942,0.368686,0.883505,0.645800,0.506705,0.323772
3,0.322695,0.328434,0.439760,0.833929,0.756416,0.215364,0.881402,0.392818,0.038540,0.428167,...,0.258877,0.376789,0.298316,0.297974,0.425308,0.851196,0.818303,0.235854,0.945321,0.303173
4,0.793706,0.484613,0.093464,0.584541,0.228840,0.362470,0.732469,0.704083,0.105755,0.346273,...,0.700124,0.542999,0.036351,0.912207,0.375398,0.331349,0.055205,0.762758,0.920264,0.385896


In [29]:
# Pause this and reflect -- do you understand the
# list comprehension and how I created this DataFrame? You
# don't have to use a list comprehension to do this,
# but you should be able to read this and figure out how it
# works as this is a commonly used approach on web forums.

In [16]:
# Ok, let's create a second dataframe
df2=pd.DataFrame([np.random.random(100) for x in range(100)])
df2.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,0.578039,0.241867,0.933837,0.104783,0.359961,0.603325,0.822837,0.920764,0.717410,0.217639,...,0.922277,0.233266,0.147578,0.356891,0.022527,0.669457,0.815692,0.851602,0.894551,0.843661
1,0.472987,0.944280,0.513955,0.610294,0.342005,0.892489,0.184800,0.826433,0.943259,0.719372,...,0.080330,0.432238,0.862486,0.600111,0.581406,0.805656,0.689755,0.863858,0.609813,0.039431
2,0.724473,0.049461,0.494578,0.948857,0.457277,0.123069,0.124595,0.980330,0.084333,0.784973,...,0.386830,0.198930,0.751839,0.583660,0.415738,0.511688,0.085685,0.486315,0.119046,0.963064
3,0.492593,0.590874,0.407702,0.701167,0.049836,0.649824,0.689006,0.150793,0.482051,0.627809,...,0.043932,0.366742,0.721899,0.172178,0.996746,0.923957,0.852845,0.223592,0.733406,0.897510
4,0.857613,0.033512,0.550982,0.925412,0.895338,0.843112,0.339575,0.998443,0.314314,0.678019,...,0.217061,0.244462,0.857014,0.697517,0.903317,0.873525,0.518331,0.089181,0.043721,0.158491


In [17]:
# Are these two DataFrames the same? 
# Maybe a better question is, for a given row inside of df1, 
# is it the same as the row inside df2?

# Let's take a look. Let's say our critical value is 0.1,
# or an alpha of 10%. And we're going to compare each
# column in df1 to the same numbered column in df2.
# And we'll report when the p-value isn't less than 10%,
# which means that we have sufficient evidence to say that 
# the columns are different.

# Let's write this in a function called test_columns
def test_columns(alpha=0.1):
    # I want to keep track of how many differ
    num_diff=0
    # And now we can just iterate over the columns
    for col in df1.columns:
        # we can run out ttest_ind between the two dataframes
        teststat,pval=ttest_ind(df1[col],df2[col])
        # and we check the pvalue versus the alpha
        if pval<=alpha:
            # And now we'll just print out if they are
            # different and increment the num_diff
            print("Col {} is statistically significantly different at alpha={}, pval={}".format(col,alpha,pval))
            num_diff=num_diff+1
    # and let's print out some summary stats
    print("Total number different was {}, which is {}%".format(num_diff,float(num_diff)/len(df1.columns)*100))

# And now lets actually run this
test_columns()

Col 0 is statistically significantly different at alpha=0.1, pval=0.016256581973286114
Col 14 is statistically significantly different at alpha=0.1, pval=0.0678700305496596
Col 21 is statistically significantly different at alpha=0.1, pval=0.09840268721577645
Col 41 is statistically significantly different at alpha=0.1, pval=0.07897481412435241
Col 45 is statistically significantly different at alpha=0.1, pval=0.06970722408909921
Col 59 is statistically significantly different at alpha=0.1, pval=0.015657061418832455
Col 62 is statistically significantly different at alpha=0.1, pval=0.06051110721640995
Col 78 is statistically significantly different at alpha=0.1, pval=0.07397709993523054
Total number different was 8, which is 8.0%


In [18]:
# Interesting, so we see that there are a bunch of columns 
# that are different! In fact, that number looks a
# lot like the alpha value we chose. So what's going on - 
# shouldn't all of the columns be the same? Remember
# that all the ttest does is check if two sets are similar
# given some level of confidence, in our case, 10%.
# The more random comparisons you do, the more will just happen
# to be the same by chance. In this example, we
# checked 100 columns, 
# so we would expect there to be roughly 10 of them if our alpha was 0.1.

# We can test some other alpha values as well
test_columns(0.05)

Col 0 is statistically significantly different at alpha=0.05, pval=0.016256581973286114
Col 59 is statistically significantly different at alpha=0.05, pval=0.015657061418832455
Total number different was 2, which is 2.0%


In [19]:
# So, keep this in mind when you are doing statistical tests 
# like the t-test which has a p-value. Understand
# that this p-value isn't magic,
# that it's a threshold for you when reporting results and trying to answer
# your hypothesis. 
#What's a reasonable threshold? Depends on your question,
# and you need to engage domain
# experts to better understand what they would consider significant.

# Just for fun, lets recreate that second dataframe 
# using a non-normal distribution, I'll arbitrarily chose
# chi squared
df2=pd.DataFrame([np.random.chisquare(df=1,size=100) for x in range(100)])
test_columns()

Col 0 is statistically significantly different at alpha=0.1, pval=0.000558682117398937
Col 1 is statistically significantly different at alpha=0.1, pval=0.0005155820441298651
Col 2 is statistically significantly different at alpha=0.1, pval=0.00022980133075183737
Col 3 is statistically significantly different at alpha=0.1, pval=1.0486726219602672e-06
Col 4 is statistically significantly different at alpha=0.1, pval=5.337104433227248e-06
Col 5 is statistically significantly different at alpha=0.1, pval=0.00011837906177713294
Col 6 is statistically significantly different at alpha=0.1, pval=4.556132614162642e-06
Col 7 is statistically significantly different at alpha=0.1, pval=0.04348699597149588
Col 8 is statistically significantly different at alpha=0.1, pval=9.892347034729827e-05
Col 9 is statistically significantly different at alpha=0.1, pval=0.0054525379965318236
Col 10 is statistically significantly different at alpha=0.1, pval=3.704669135279052e-05
Col 11 is statistically signifi

In [39]:
# Now we see that all or most columns test to be
#statistically significant at the 10% level.

In this lecture, we've discussed just some of the basics of hypothesis testing in Python. I introduced you
to the SciPy library, which you can use for the students t test. We've discussed some of the practical
issues which arise from looking for statistical significance. There's much more to learn about hypothesis
testing, for instance, there are different tests used, depending on the shape of your data and different
ways to report results instead of just p-values such as confidence intervals or bayesian analyses. But this
should give you a basic idea of where to start when comparing two populations for differences, which is a
common task for data scientists.